In [104]:
# Notes/ Ideas
# input data = can calculate avg. over time period, average finish over time 
# determine metric to predict
# find OWGR algorithm and compare
# linear regression only works on numeric metrics, so we need to figure out a way to add information about tournament strength
# explore data to add to tornaments: major indicator, strength of field, etc.
# explore the idea of using Scotty as a 'model' and predicting what other golfers are on similar trajectory. need to include avg. finish over a time period.
# potentially calculate golfer-level statistics 

# idea: instead of directly linking tournament to ranking, look at a range.
# for example, scottie shefler trajectory over 3/1 to 5/1 would include all OWGR ranks as a sequence
# we can then look at stats (SG average, SG Rank) and finishes. We can compare sequences and predict rankings rise, performance ranking, etc. 

In [105]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [106]:
# import PGA data
pga_df = pd.read_csv('PGA_Data.csv')

# import OWGR data
owgr_df = pd.read_csv('OWGR.csv')

In [107]:
# drop fantasy related and filler columns
pga_df = pga_df.drop(columns=['hole_DKP', 'hole_FDP', 'hole_SDP', 'streak_DKP',
       'streak_FDP', 'streak_SDP', 'n_rounds', 'made_cut', 'pos', 'finish_DKP',
       'finish_FDP', 'finish_SDP', 'total_DKP', 'total_FDP', 'total_SDP',
       'player', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',])

In [108]:
# make a column that is the finish, removing ties and turning 'CUT' to 0. 

# 0 could be changed to 99, NA, or whatever the cut line is (count in tournament / 2)

# note, the data column is an 'Object' type so it has strings and integers. 
# this methods turns everything to a string, makes its caclualtions, then turns to a float so we can do statistical calculations.

def adjust_finish(x):
    # this treats cut, dq, wd, etc. all as the same and assigns a value of 99
    # todo: determine if we want to treat these all the same
    # todo: because lower is better, NFs are treated as 99, but we can hone this in, 
    if x == 'CUT' or x == 'DQ' or x == 'WD' or x == 'MDF' or x == 'W/D':
        return '99'
    elif x.startswith('T'):
        return x[1:]
    else:
        return x
pga_df['finish_adj'] = pga_df['Finish'].astype(str).apply(adjust_finish).astype(float)


In [114]:
# calculate score to par
pga_df['score'] = pga_df['strokes'] - pga_df['hole_par']

# turn names uppercase for matching with OWGR
pga_df['Player_initial_last'] = pga_df['Player_initial_last'].str.upper()

# rename date column for merging
pga_df = pga_df.rename(columns={'date': 'Date'})


In [110]:
# what does Scottie's stats look like?

pga_df[pga_df['Player_initial_last'] == 'S. SCHEFFLER'].head(50)

,Player_initial_last,tournament id,player id,hole_par,strokes,tournament name,course,date,purse,season,no_cut,Finish,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total,finish_adj,score
52,S. SCHEFFLER,401353275,9478,284,273,BMW Championship,"Wilmington Country Club - Wilmington, DE",2022-08-21,15.00,2022,1,T3,0.26,-0.17,1.36,0.55,1.74,2.00,3.0,-11
159,S. SCHEFFLER,401353274,9478,140,139,FedEx St. Jude Classic,"TPC Southwind - Memphis, TN",2022-08-14,15.00,2022,0,CUT,-1.95,0.78,-0.46,1.11,1.44,-0.51,99.0,-1
648,S. SCHEFFLER,401353217,9478,288,279,The Open,"St. Andrews - St. Andrews, Scotland",2022-07-17,14.00,2022,0,T21,-0.42,0.10,1.20,0.72,2.02,1.61,21.0,-9
842,S. SCHEFFLER,401353219,9478,140,145,Scottish Open,"The Renaissance Club - North Berwick, Scotland",2022-07-10,8.00,2022,0,CUT,-0.63,-1.06,0.66,0.81,0.41,-0.22,99.0,5
1114,S. SCHEFFLER,401353221,9478,280,270,Travelers Championship,"TPC River Highlands - Cromwell, CT",2022-06-26,8.30,2022,0,T13,-0.25,0.58,0.43,1.08,2.10,1.85,13.0,-10
1218,S. SCHEFFLER,401353222,9478,280,275,U.S. Open,"The Country Club - Brookline, MA",2022-06-19,17.50,2022,0,T2,0.84,-0.39,2.30,1.10,3.01,3.85,2.0,-5
1339,S. SCHEFFLER,401353223,9478,280,273,RBC Canadian Open,"St. George's Golf & Country Club - Toronto, On...",2022-06-12,8.70,2022,0,T18,-0.14,0.56,1.00,0.60,2.17,2.03,18.0,-7
1562,S. SCHEFFLER,401353225,9478,280,271,Charles Schwab Challenge,"Colonial - Ft. Worth, TX",2022-05-29,8.40,2022,0,2,0.14,0.58,1.37,1.03,2.98,3.12,2.0,-9
1689,S. SCHEFFLER,401353226,9478,140,146,PGA Championship,"Southern Hills Country Club - Tulsa, OK",2022-05-22,15.00,2022,0,CUT,-0.55,-0.38,1.20,-0.55,0.27,-0.28,99.0,6
1828,S. SCHEFFLER,401353227,9478,288,269,AT&T Byron Nelson,"TPC Craig Ranch - McKinney, TX",2022-05-15,9.10,2022,0,T15,-0.40,0.49,1.41,0.29,2.18,1.78,15.0,-19


In [111]:
# look at a tournament and examine scoring stats with ranked leaderboad.

# initial findings after research: sg total is highly correlated with victory. but the best putter doesn't always win, for example.
# determining a model that weighs each of these stats to predict finish_adj is the early stages of preidction model, for each tournament.
pga_df[pga_df['date'] ==  '2020-08-30'][['Player_initial_last', 'sg_putt','sg_arg','sg_app',	'sg_ott','sg_t2g','sg_total','finish_adj']].sort_values('sg_putt', ascending= False)

,Player_initial_last,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total,finish_adj
10756,B. TODD,1.74,0.14,-0.31,0.00,-0.17,1.57,8.0
10753,B. HARMAN,1.27,0.03,-0.07,-0.16,-0.20,1.07,12.0
10807,T. FINAU,1.15,0.49,0.15,0.27,0.92,2.07,5.0
10760,C. HOWELL III,1.12,-0.35,-0.25,-0.95,-1.56,-0.43,48.0
10752,B. DECHAMBEAU,1.11,-0.57,-1.35,0.12,-1.79,-0.68,50.0
...,...,...,...,...,...,...,...,...
10786,M. HOMA,-0.96,-0.07,-0.53,0.14,-0.47,-1.43,59.0
10767,D. LEE,-1.10,0.11,1.34,-0.29,1.17,0.07,33.0
10782,K. STREELMAN,-1.33,-0.36,1.21,-0.45,0.40,-0.93,51.0
10759,C. CONNERS,-1.95,0.52,0.87,0.62,2.01,0.07,33.0


In [112]:
# Exploring OWGR data set, creating a name column that matches the PGA dataset
# split name into first and last names
owgr_df[['first_name', 'last_name']] = owgr_df['Player'].str.split(' ', 1, expand=True)

# get the first initial of the first name
owgr_df['first_initial'] = owgr_df['first_name'].str[0]

# concatenate the first initial and last name
owgr_df['Player_initial_last'] = owgr_df['first_initial'] + '. ' + owgr_df['last_name']

# drop the intermediate columns
owgr_df = owgr_df.drop(['first_name', 'first_initial', 'last_name'], axis=1)

owgr_df[owgr_df['Date'] == '2020-08-30']



,Player,Country,OWGR,Average Points,Total Points,Events Played,Date,Player_initial_last
2666,AARON RAI,ENGLAND,238,0.7013,36.469,52,2020-08-30,A. RAI
2822,AARON WISE,UNITED STATES,207,0.8002,38.409,48,2020-08-30,A. WISE
2910,ABRAHAM ANCER,MEXICO,26,3.8630,200.875,52,2020-08-30,A. ANCER
3456,ADAM HADWIN,CANADA,66,2.0438,100.149,49,2020-08-30,A. HADWIN
3724,ADAM LONG,UNITED STATES,74,1.8399,95.676,52,2020-08-30,A. LONG
...,...,...,...,...,...,...,...,...
313359,ZAC BLAIR,UNITED STATES,231,0.7241,37.651,52,2020-08-30,Z. BLAIR
314248,ZACH JOHNSON,UNITED STATES,179,0.9423,43.346,46,2020-08-30,Z. JOHNSON
314374,ZACK SUCHER,UNITED STATES,219,0.7548,30.192,40,2020-08-30,Z. SUCHER
314459,ZANDER LOMBARD,SOUTH AFRICA,253,0.6616,34.404,52,2020-08-30,Z. LOMBARD


In [126]:
# lets just try a merge and see how many matches we get with OWGR and Tournament.
merged_df = pd.merge(pga_df,owgr_df,on=['Player_initial_last','Date'])

# perfect matches = 19,000. need to figure out left join that has high results

In [127]:
merged_df[merged_df['Date'] == '2020-08-30']


,Player_initial_last,tournament id,player id,hole_par,strokes,tournament name,course,Date,purse,season,...,sg_t2g,sg_total,finish_adj,score,Player,Country,OWGR,Average Points,Total Points,Events Played
2837,A. ANCER,401155475,9261,280,287,BMW Championship,"Olympia Fields Country Club - Olympia Fields, IL",2020-08-30,9.5,2020,...,-0.79,0.07,33.0,7,ABRAHAM ANCER,MEXICO,26,3.8630,200.875,52
2838,A. HADWIN,401155475,5548,280,288,BMW Championship,"Olympia Fields Country Club - Olympia Fields, IL",2020-08-30,9.5,2020,...,0.40,-0.18,40.0,8,ADAM HADWIN,CANADA,66,2.0438,100.149,49
2839,A. LANDRY,401155475,4682,280,301,BMW Championship,"Olympia Fields Country Club - Olympia Fields, IL",2020-08-30,9.5,2020,...,-3.03,-3.43,68.0,21,ANDREW LANDRY,UNITED STATES,122,1.3246,68.880,52
2840,A. LONG,401155475,6015,280,292,BMW Championship,"Olympia Fields Country Club - Olympia Fields, IL",2020-08-30,9.5,2020,...,-1.31,-1.18,56.0,12,ADAM LONG,UNITED STATES,74,1.8399,95.676,52
2841,A. NOREN,401155475,3832,280,288,BMW Championship,"Olympia Fields Country Club - Olympia Fields, IL",2020-08-30,9.5,2020,...,-0.36,-0.18,40.0,8,ALEX NOREN,SWEDEN,89,1.5805,82.187,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2896,T. HATTON,401155475,5553,280,284,BMW Championship,"Olympia Fields Country Club - Olympia Fields, IL",2020-08-30,9.5,2020,...,1.16,0.82,16.0,4,TYRRELL HATTON,ENGLAND,15,4.6305,222.265,48
2897,T. HOGE,401155475,6086,280,295,BMW Championship,"Olympia Fields Country Club - Olympia Fields, IL",2020-08-30,9.5,2020,...,-1.76,-1.93,65.0,15,TOM HOGE,UNITED STATES,145,1.1788,61.295,52
2898,T. WOODS,401155475,462,280,291,BMW Championship,"Olympia Fields Country Club - Olympia Fields, IL",2020-08-30,9.5,2020,...,-0.06,-0.93,51.0,11,TIGER WOODS,UNITED STATES,18,4.2844,171.375,40
2899,V. HOVLAND,401155475,4364873,280,288,BMW Championship,"Olympia Fields Country Club - Olympia Fields, IL",2020-08-30,9.5,2020,...,-0.55,-0.18,40.0,8,VIKTOR HOVLAND,NORWAY,31,3.2911,131.643,40


In [128]:
# starting to build a model that predicts the finish, for one tournament to start, the most recent bmw championship.
# will use linear regression


# lets try on the whole data set, must drop NaNs from this data
pga_df = pga_df.dropna(subset=['finish_adj','sg_putt',	'sg_arg',	'sg_app',	'sg_ott'])
merged_df = merged_df.dropna(subset=['finish_adj','sg_putt',	'sg_arg',	'sg_app',	'sg_ott'])
# drop WDs, cuts, etc.
pga_df = pga_df[pga_df.finish_adj != 99]
merged_df = merged_df[merged_df.finish_adj != 99]

In [136]:
# X = elements of model
# Y = predicted value
X = merged_df[['sg_putt',	'sg_arg',	'sg_app',	'sg_ott']]
y = merged_df['finish_adj']

# Split the data into a training set and a testing set. 
# The training set will be used to train the model, and the testing set will be used to evaluate the model's performance.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [137]:
# Create a LinearRegression object and fit it to the training data.

lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression()

In [138]:
y_pred = lm.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Mean squared error:', mse)
print('R-squared:', r2)
print('Coefficents:', lm.coef_)

Mean squared error: 116.9808079558773
R-squared: 0.7459175620703602
Coefficents: [-13.63722371 -13.85716689 -13.61659794 -13.67423225]


In [140]:
# model has been trained on entire PGA training set. lets see how the results compare to the bmw championship
bmw = pga_df[pga_df['tournament id'] ==  401353275][['tournament id','player id','Player_initial_last', 'sg_putt','sg_arg','sg_app',	'sg_ott','sg_t2g','sg_total','finish_adj']].dropna(subset=['finish_adj'])


# attach a 'predicted' column that, based on the 4 parameters (these match the training parameters), that will predict the finish
# bmw['predicted'] = lm.predict(bmw[['sg_putt','sg_arg','sg_app',	'sg_ott']])

# print bmw dataframe that has player id, what their actual finish is, and what, based on their scoring stats, their predicted finish would be
# sort by predicted to compare. 
#bmw[['player id', 'finish_adj','predicted']].sort_values('predicted').head(50)

# it works pretty well, identifies the winner but struggles with places 8 thru 15. not sure how valuable this is but it is a start. 

# trying this on the entire dataset
pga_df['predicted'] = lm.predict(pga_df[['sg_putt','sg_arg','sg_app','sg_ott']])
pga_df[['tournament id', 'player id', 'finish_adj', 'score','predicted']].sort_values('predicted').head(50)

# interesting results, tournament id 401243416 is the memorial tournament where jon rahm, who was leading, WD because of a covid test.


,tournament id,player id,finish_adj,score,predicted
35600,2245,1112,1.0,-27,-72.050801
35570,2245,446,2.0,-23,-44.645578
10847,401155474,3448,1.0,-30,-36.063907
3522,401353235,6086,1.0,-19,-33.760427
13347,401155419,9261,2.0,-24,-33.700972
37032,2233,1067,10.0,-15,-33.429011
3727,401353233,5504,1.0,-23,-32.060509
10439,401219333,10046,1.0,-6,-31.889126
13399,401155419,5462,10.0,-18,-31.735938
35556,2245,5025,55.0,-12,-31.210130


In [ ]:
# looking at some results, we will need to come up with a better way to handle CUTs and WDs. 

In [141]:
pga_df[pga_df['tournament id'] == 401155419].sort_values('finish_adj')

,Player_initial_last,tournament id,player id,hole_par,strokes,tournament name,course,Date,purse,season,...,Finish,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total,finish_adj,score,predicted
13352,A. LANDRY,401155419,4682,288,262,The American Express,"La Quinta CC - La Quinta, CA",2020-01-18,6.7,2020,...,1,1.81,-0.90,1.94,1.18,2.22,4.03,1.0,-26,-10.547119
13347,A. ANCER,401155419,9261,288,264,The American Express,"La Quinta CC - La Quinta, CA",2020-01-18,6.7,2020,...,2,2.38,0.89,1.74,0.69,3.32,5.70,2.0,-24,-33.700972
13480,S. SCHEFFLER,401155419,9478,288,265,The American Express,"La Quinta CC - La Quinta, CA",2020-01-18,6.7,2020,...,3,0.48,0.10,0.50,1.12,1.72,2.20,3.0,-23,14.161576
13482,S. STRAKA,401155419,8961,288,268,The American Express,"La Quinta CC - La Quinta, CA",2020-01-18,6.7,2020,...,T4,1.75,-0.56,2.78,0.92,3.14,4.89,4.0,-20,-22.322965
13360,B. CAULEY,401155419,5338,288,268,The American Express,"La Quinta CC - La Quinta, CA",2020-01-18,6.7,2020,...,T4,1.10,-0.11,0.62,0.59,1.10,2.20,4.0,-20,14.229854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13489,T. MERRITT,401155419,3970,288,281,The American Express,"La Quinta CC - La Quinta, CA",2020-01-18,6.7,2020,...,T68,-1.44,-1.23,0.37,0.33,-0.53,-1.97,68.0,-7,71.347879
13404,H. NORLANDER,401155419,5573,288,281,The American Express,"La Quinta CC - La Quinta, CA",2020-01-18,6.7,2020,...,T68,-0.15,-0.61,-1.57,-1.64,-3.82,-3.97,68.0,-7,98.518854
13372,B. STUARD,401155419,3599,288,282,The American Express,"La Quinta CC - La Quinta, CA",2020-01-18,6.7,2020,...,72,-0.97,0.51,-0.76,-0.89,-1.14,-2.11,72.0,-6,72.896232
13351,A. LAHIRI,401155419,4989,288,283,The American Express,"La Quinta CC - La Quinta, CA",2020-01-18,6.7,2020,...,73,-0.42,-0.42,-1.75,-0.53,-2.69,-3.11,73.0,-5,86.840633
